<a href="https://colab.research.google.com/github/gustabart/dm/blob/main/competencia_udemy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import pandas as pd

import sklearn as sk
from sklearn import model_selection
from sklearn import ensemble
from sklearn import metrics

# Lectura de datos

In [ ]:
df_entrenamiento = pd.read_excel('datasets/udemy_entrenamiento.xlsx', index_col="id")
df_entrenamiento.head()

In [ ]:
df_prueba = pd.read_excel('datasets/udemy_prueba.xlsx', index_col="id")
df_prueba.head()

# Entendimiento (magia parte 1)

In [ ]:
df_entrenamiento['rating'].hist();

In [ ]:
df_entrenamiento['locale'].value_counts()

In [ ]:
df_entrenamiento.corr()

In [ ]:
df_entrenamiento.info()

# Limpieza (magia parte 2)

In [62]:
columnas = ['is_practice_test_course', 'rating', 'num_published_lectures', 'num_published_practice_tests', 'bestseller']
df_entrenamiento = df_entrenamiento[columnas]
df_prueba = df_prueba[columnas]

In [ ]:
# Tips:
# select_dtypes (Para hacer la selección de columnas mas elegante)
# map, replace, para reemplazar datos
# loc
# df_entrenamiento.loc[df_entrenamiento['bestseller'] == "yes", 'bestseller'] = "si"
# df['bestseller'].unique()
# df['bestseller'].value_counts() 
# usar dummies para transformar categorías

In [64]:
# Reemplaza los no por los yes
df_entrenamiento["bestseller"] = df_entrenamiento["bestseller"].map({"no":"no", "yes":"si", "si":"si"})
df_entrenamiento["bestseller"].value_counts()

no    8887
si    2200
Name: bestseller, dtype: int64

In [ ]:
# Limpiar $ de los precios

In [ ]:
# Hago los cambios en df_prueba también
df_entrenamiento["bestseller"] = df_entrenamiento["bestseller"].map({"no":"no", "yes":"si", "si":"si"})
df_prueba["bestseller"].value_counts()

# Territorio de Aprendizaje Automático
Se mira y no se toca

In [ ]:
# Datos para probar
X = df_entrenamiento[df_entrenamiento.columns.drop('bestseller')]
y = df_entrenamiento['bestseller']

# Partimos en entrenamiento y prueba 
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Creamos el objeto del modelo
clf = sk.ensemble.RandomForestClassifier(n_estimators=3000, criterion='gini', max_depth=5, n_jobs=-1, random_state=42)

# Entrenamos el modelo
clf.fit(X_train, y_train)

# Predecimos
y_pred = clf.predict(X_test)

# Medimos la performance de la predicción
sk.metrics.accuracy_score(y_test, y_pred)

In [ ]:
# Datos a predecir 
X_prueba = df_prueba[df_prueba.columns.drop('bestseller')]

# Entrenamos el modelo con todos los datos
clf.fit(X, y)

# Predecimos
df_prueba['bestseller'] = clf.predict(X_prueba)

# Grabamos
df_prueba['bestseller'].to_csv('datasets/solucion.csv')